# Word Embeddings

The large number of English words can make language-based applications daunting. To cope with this, it is helpful to have a clustering or embedding of these words, so that words with similar meanings are clustered together, or have embeddings that are close to one another. That is, words that tend to appear in similar contexts are likely to be related.

## Import libraries

In [2]:
import nltk
import numpy as np
from nltk.corpus import stopwords
from string import punctuation
from collections import defaultdict
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
#nltk.download('brown')

## Pr(c|w) & Pr(c)

1. Remove stopwords and punctuation, make everything lowercase, and count how often each word occurs.
Use this to come up with two lists:
\begin{itemize}
\item A vocabulary V , consisting of a few thousand (e.g., 5000) of the most commonly-occurring words.
\item A shorter list C of at most 1000 of the most commonly-occurring words, which we shall call
context words.
\end{itemize}
   
2. For each word $w \in V$, and each occurrence of it in the text stream, look at the surrounding window of
four words (two before, two after): $w_1, w_2, w, w_3, w_4$. Keep count of how often context words from C appear in these positions around word w. That is, for
$w \in V$, $c \in C$, define:
                   
                   n(w, c) = # of times c occurs in a window around w
Using these counts, construct the probability distribution Pr(c|w) of context words around w (for each $w \in V$), as well as the overall distribution Pr(c) of context words. These are distributions over C.
    
3. Represent each vocabulary item w by a |C|-dimensional vector $\Phi(w)$, whose c'th coordinate is:
\begin{equation*}
\Phi_c(w) = max(0, log\dfrac{Pr(c|w)}{Pr(c)})
\end{equation*}
This is known as the (positive) pointwise mutual information, and has been quite successful in work
on word embeddings.

In [3]:
words = nltk.corpus.brown.words()
filteredWords = []
wordCount = defaultdict(int)

for word in words:
    if word.lower() not in set(stopwords.words('english')):
        w = ''.join([c for c in word.lower() if c not in set(punctuation)])
        if w != '':
            wordCount[w] += 1
            filteredWords.append(w)

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

V = [count[1] for count in counts[:5000]] 
C = [count[1] for count in counts[:1000]]

#Pr(c)
Pr_c = defaultdict(float)
for c in C:
    Pr_c[c] = wordCount[c] * 1.0 / sum(wordCount[c] for c in C)
    
wWindow = defaultdict(list)
for i in xrange(0, len(filteredWords)):
    if filteredWords[i] in V:
        if i == 0:
            wWindow[filteredWords[i]].extend(neigh for neigh in [filteredWords[i+1], filteredWords[i+2], filteredWords[i+3], filteredWords[i+4]] if neigh in C)
        if i == 1:
            wWindow[filteredWords[i]].extend(neigh for neigh in [filteredWords[i-1], filteredWords[i+1], filteredWords[i+2], filteredWords[i+3]] if neigh in C)
        if 2 <= i <= len(filteredWords) - 3:
            wWindow[filteredWords[i]].extend(neigh for neigh in [filteredWords[i-2], filteredWords[i-1], filteredWords[i+1], filteredWords[i+2]] if neigh in C)
        if i == len(filteredWords) - 2:
            wWindow[filteredWords[i]].extend(neigh for neigh in [filteredWords[i+1], filteredWords[i-1], filteredWords[i-2], filteredWords[i-3]] if neigh in C)
        if i == len(filteredWords) - 1:
            wWindow[filteredWords[i]].extend(neigh for neigh in [filteredWords[i-1], filteredWords[i-2], filteredWords[i-3], filteredWords[i-4]] if neigh in C)

#Pr(c|w)
nwc = defaultdict(int)
for w in V:
    for c in C:
        nwc[(w, c)] = wWindow[w].count(c)
        #Pr_cw[i, j] = nwc[(V[i], C[j])] * 1.0 / len(wWindow[V[i]])
        
def Pr_cw(c, w):
    return nwc[(w, c)] * 1.0 / len(wWindow[w]) #Pr_c[c]

phi_c = defaultdict(list)
for w in V:
    phi_c[w] = [max(0, np.log2(Pr_cw(c, w) * 1.0 / Pr_c[c])) for c in C]

/Users/bowen/miniconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: divide by zero encountered in log2


## PCA
a 100-dimensional representation

In [6]:
k, v = list(zip(*phi_c.items()))
pca = PCA(n_components=100,svd_solver='full') #auto -> randomized
v_rd = pca.fit_transform(v)
phi_c_rd = dict(zip(k, v_rd))

## NN

Pick a collection of 25 words $w \in V$. For each w, return its nearest neighbor $w'\neq w$ in V . A popular distance measure to use for this is cosine distance:
\begin{equation*}
1 - \dfrac{\Psi(w)\cdot\Psi(w')}{||\Psi(w)||\cdot||\Psi(w')||}
\end{equation*}


In [9]:
def distance(wa, wb):
    return 1 - phi_c_rd[wa].dot(phi_c_rd[wb]) * 1.0 / (np.linalg.norm(phi_c_rd[wa]) * np.linalg.norm(phi_c_rd[wb]))

def nn(word, Vacabulary):
    allDistance = []
    for w in Vacabulary:
        if w != word:
            allDistance.append(distance(word, w)) 
    if np.argmin(allDistance) >= Vacabulary.index(word):
        return Vacabulary[np.argmin(allDistance)+1]
    else:
        return Vacabulary[np.argmin(allDistance)]

# nn = NearestNeighbors(n_neighbors=2, metric='cosine')
# nn.fit(V)
# nn.kneighbors(phi_c_rd['communism'].reshape(1,-1))

In [10]:
wordC = ['communism', 'autumn', 'cigarette', 'pulmonary', 'mankind', 'africa', 'chicago', 'revolution', 'september', 'chemical', 'detergent', 'dictionary', 'storm', 'worship', 'friday', 'jury', 'election', 'widespread', 'considering', 'size', 'excuse', 'governments', 'date', 'major', 'money']
nNeighs = []

for word in wordC:
    nNeighs.append(nn(word, k))

print nNeighs

# NN = NearestNeighbors(n_neighbors=2, metric='cosine')
# NN.fit(V)
# NN.kneighbors(phi_c_rd['communism'].reshape(1,-1))

[u'utopian', u'summer', u'bullet', u'artery', u'world', u'asia', u'portland', u'intellectual', u'july', u'drugs', u'fabrics', u'text', u'saturday', u'religious', u'sunday', u'courts', u'committee', u'liberals', u'useful', u'measuring', u'policeman', u'government', u'15', u'problems', u'pay']


The nearest neighbors of some words are semantically related with them
or belong to the same category.

## Clustering
Using the vectorial representation $\Psi(\cdot)$, cluster the words in V into 100 groups.

### GM

In [11]:
gm = GaussianMixture(n_components=100)
gm.fit(v_rd)
groups = gm.predict(v_rd)

clusters = defaultdict(list)
for i in xrange(0, len(groups)):
    clusters[groups[i]].append(k[i])

In [12]:
smallClusters = []
for i in xrange(0, len(clusters)):
    if len(clusters[i]) <= 15:
        smallClusters.append(clusters[i])
        
smallClusters

[[u'saturday', u'pm', u'am', u'tuesday'],
 [u'standard',
  u'directly',
  u'provided',
  u'prices',
  u'apply',
  u'institution',
  u'sources'],
 [u'henry',
  u'john',
  u'george',
  u'mrs',
  u'director',
  u'richard',
  u'robert',
  u'thomas',
  u'william'],
 [u'street', u'side', u'across', u'road', u'river'],
 [u'1', u'2', u'3', u'4', u'5', u'6', u'7', u'8'],
 [u'memorial',
  u'mayor',
  u'miami',
  u'st',
  u'louis',
  u'author',
  u'greenwich',
  u'governor',
  u'chairman',
  u'maryland',
  u'houston',
  u'atlanta',
  u'francisco'],
 [u'direct', u'method', u'laws', u'described', u'analysis']]

### k-Means

In [13]:
km = KMeans(n_clusters=100).fit(v_rd)
groupsKM = km.labels_

clustersKM = defaultdict(list)
for i in xrange(0, len(groupsKM)):
    clustersKM[groupsKM[i]].append(k[i])

smallClustersKM = []
for i in xrange(0, len(clustersKM)):
    if len(clustersKM[i]) <= 15:
        smallClustersKM.append(clustersKM[i])
        
smallClustersKM

[[u'todays', u'standards'],
 [u'india',
  u'entitled',
  u'shall',
  u'commission',
  u'commerce',
  u'legislation',
  u'title',
  u'act',
  u'directed',
  u'authorized',
  u'sec',
  u'provision',
  u'provisions'],
 [u'values', u'behavior', u'value'],
 [u'daily', u'14', u'6', u'24', u'feed'],
 [u'l', u'n', u'p', u'r', u'b', u'c', u'e', u'f', u'g', u'j'],
 [u'division', u'uses', u'follows', u'requires', u'establishment'],
 [u'wednesday',
  u'saturday',
  u'reception',
  u'sunday',
  u'thursday',
  u'friday',
  u'scheduled',
  u'yesterday',
  u'am',
  u'oclock',
  u'tuesday',
  u'monday']]

The Gaussian Mixture model was selected to fit the data into 100 groups rather than K-Means, because
besides of the means, the covariance information (important to this word embeddings clustering case)
of the data is also utilized, which could generate more generalized clusters to some extent. The expectation-maximization algorithm is implemented in its learning process, which firstly initializes
random components and repeatedly assigns each point a probability between the clusters and then
updates the parameters including weights, means and covariances to maximize the likelihood given
the assignments until convergence. The distance function involved in is the Mahalanobis distances
between points and centers.